In [1]:
import dis
def get_code_hash(func):
    code_set = set() # memoization set
    def _traverse_code(analyzed_function):
        inner_functions_set = set()
        instructions = list(dis.get_instructions(analyzed_function))
        instructions = list(reversed(instructions))

        for ix, inst in enumerate(instructions):
            if 'CALL_FUNCTION' == inst.opname:
                called_function_name = instructions[ix + inst.arg + 1]
                function_name = called_function_name.argval
                func = globals()[function_name]
                if func not in code_set:
                    code_set.add(func)
                    inner_functions_set.add(func)
        code_list = [_traverse_code(f) for f in inner_functions_set]
        code_list.append(analyzed_function.__code__.co_code)
        if len(code_list) == 1:
            return code_list[0]
        else:
            return code_list
    bytecode_list = _traverse_code(func)
    if isinstance(bytecode_list,list):
        return hash(b''.join(bytecode_list))
    else:
        return hash(bytecode_list)

In [2]:
def a(x):
    return x+b(x)
def b(x):
    return x+1
def c(z):
    return z+b(z)
def d(x):
    return x + d(x)

In [3]:
test_set = [a,b,c,d]
[get_code_hash(f) for f in test_set]

[3287583857289421461,
 -3548610114888071795,
 3287583857289421461,
 -2918565914108623256]